In [1]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/synthetic_traffic_data.csv')

print(df.head())

Mounted at /content/drive
             timestamp  density_north  density_south  density_east  \
0  2023-01-01 00:00:00       0.092116       0.434155      0.206124   
1  2023-01-01 00:01:00       0.008046       0.426411      0.216333   
2  2023-01-01 00:02:00       0.055668       0.499161      0.280999   
3  2023-01-01 00:03:00       0.004533       0.457348      0.224563   
4  2023-01-01 00:04:00       0.080436       0.425959      0.229743   

   density_west  
0      0.279817  
1      0.266353  
2      0.284039  
3      0.270084  
4      0.249114  


In [2]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

df['current_green_direction'] = df[['density_north', 'density_south', 'density_east', 'density_west']].idxmax(axis=1).str.replace('density_', '')
df['current_green_duration'] = (df[['density_north', 'density_south', 'density_east', 'density_west']].max(axis=1) * 60).astype(int)

df['next_green_direction'] = df['current_green_direction'].shift(-1).fillna('north')
df['next_green_duration'] = df['current_green_duration'].shift(-1).fillna(30)

df = df.iloc[:-1]

In [3]:
import numpy as np

df['hour'] = df['timestamp'].dt.hour
df['time_sin'] = np.sin(2 * np.pi * df['hour']/24)
df['time_cos'] = np.cos(2 * np.pi * df['hour']/24)

direction_map = {'north':0, 'south':1, 'east':2, 'west':3}
df['next_green_encoded'] = df['next_green_direction'].map(direction_map)

duration_max = 60
df['duration_normalized'] = df['next_green_duration'] / duration_max

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

features = ['time_sin', 'time_cos', 'density_north', 'density_south',
            'density_east', 'density_west', 'current_green_duration']
X = df[features]


y_dir = df['next_green_encoded']
y_dur = df['duration_normalized']


X_train, X_test, y_dir_train, y_dir_test, y_dur_train, y_dur_test = train_test_split(
    X, y_dir, y_dur, test_size=0.2, random_state=42
)


scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))


x = LSTM(64, return_sequences=True)(inputs)
x = LSTM(32)(x)


direction_out = Dense(4, activation='softmax', name='direction')(x)
duration_out = Dense(1, activation='sigmoid', name='duration')(x)

model = Model(inputs=inputs, outputs=[direction_out, duration_out])

model.compile(
    optimizer='adam',
    loss={
        'direction': 'sparse_categorical_crossentropy',
        'duration': 'mse'
    },
    metrics={'direction': 'accuracy'}
)

In [6]:
history = model.fit(
    X_train,
    {'direction': y_dir_train, 'duration': y_dur_train},
    epochs=50,
    batch_size=32,
    validation_data=(X_test, {'direction': y_dir_test, 'duration': y_dur_test}))

Epoch 1/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - direction_accuracy: 0.8019 - direction_loss: 1.3233 - duration_loss: 0.0231 - loss: 1.3463 - val_direction_accuracy: 0.9028 - val_direction_loss: 1.0148 - val_duration_loss: 0.0208 - val_loss: 1.0356
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - direction_accuracy: 0.8909 - direction_loss: 0.8297 - duration_loss: 0.0169 - loss: 0.8466 - val_direction_accuracy: 0.9028 - val_direction_loss: 0.4007 - val_duration_loss: 0.0084 - val_loss: 0.4091
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - direction_accuracy: 0.8958 - direction_loss: 0.4043 - duration_loss: 0.0092 - loss: 0.4134 - val_direction_accuracy: 0.9028 - val_direction_loss: 0.3504 - val_duration_loss: 0.0087 - val_loss: 0.3591
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - direction_accuracy: 0.8933 - direction_loss: 0.3713 - duration_loss: 0.0092 - loss: 0.3804 - val_direction_accuracy: 0.9028 - val_direction_loss: 0.3253 - val_duration_loss: 0.0087 - val_loss

In [7]:
def predict_phase(current_state):
    """
    current_state format:
    {
        'time': '08:30:00',  # String or datetime
        'densities': [0.7, 0.6, 0.4, 0.5],  # N,S,E,W
        'current_duration': 45  # Current green duration
    }
    """

    time = pd.to_datetime(current_state['time'])
    features = np.array([
        np.sin(2 * np.pi * time.hour/24),
        np.cos(2 * np.pi * time.hour/24),
        *current_state['densities'],
        current_state['current_duration']
    ]).reshape(1, 1, -1)


    dir_probs, dur_norm = model.predict(features)


    directions = ['north', 'south', 'east', 'west']
    next_dir = directions[np.argmax(dir_probs)]
    next_dur = int(dur_norm[0][0] * 60)

    return {'direction': next_dir, 'duration': max(15, min(60, next_dur))}

# Example
print(predict_phase({
    'time': '08:30:00',
    'densities': [0.8, 0.6, 0.3, 0.4],
    'current_duration': 30
}))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
{'direction': 'south', 'duration': 48}
